In [27]:
from src.lib import *
from src.transfomer import *

## Data Prepocessing

In [28]:
with open("./data/poem.txt",'r') as f:
    text = f.read()

In [29]:
"".join(list(set(text)))

'JqHvjTp.hQsR;:\xad“yºmw2!Eï eBoxua¡I”1U\u200aAS8C"MVzlDd—-0ink&3?³Nbc)gfW(KÃt½P\nFrL,Z’¿G\'O\u2009Y'

In [31]:
text = text.lower()

In [32]:
data = re.split('(\W)',text)
# data = text.split(" ")

In [33]:
for d in data:
    if d=="" or d==" ":
        data.remove(d)

In [38]:
chars = sorted(list(set(data)))

In [39]:
vocab_size = len(chars)

In [40]:
vocab_size

3781

In [ ]:
str_to_idx = {char:idx for idx,char in enumerate(chars)}
idx_to_str = {idx:char for idx,char in enumerate(chars)}

In [ ]:
encode = lambda s :[str_to_idx[i] for i in s]
decode = lambda l : " ".join([idx_to_str[i] for i in l ])

In [ ]:
text_encoded = encode(data)
text_encoded = torch.tensor(text_encoded)

In [ ]:
data[:6][2]

In [ ]:
print(decode(encode(data[:20])))

In [ ]:
p = 0.9
n = int(p * len(text_encoded))
train_data = text_encoded[0:n]
test_data = text_encoded[n:]

In [ ]:
train_data.shape,test_data.shape

## Embedding

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = TransformerModel(
    vocab_size=vocab_size,
    embed_size=384,
    heads=6,
    num_layers=6,
    max_len=1000,
    device=device,
    decode_vocab=decode,
    dropout=0.2,
    lr=2e-4
)

In [ ]:
print(sum(p.numel() for p in model.parameters()))

In [ ]:
decode(train_data[:10].tolist())

In [ ]:
model.fit(train_data[:1000],batch_size=1,block_size=256,n_iter=1000,print_every=20)

In [ ]:
context = torch.zeros((1,1),dtype=torch.long,device=device)
# context = train_data[:50].reshape(1,-1).to(device)
model.inference(context, max_token=10)

In [ ]:
model.plot()